In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from tqdm import tqdm
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose


In [2]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)  
    return pose

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [3]:
# 가져올 떄
train_df=pd.read_csv('human_act_train_vid.csv')
valid_df=pd.read_csv('human_act_valid_vid.csv')

In [4]:
import os
os.mkdir('human_act_skeleton_npy')
os.mkdir('human_act_skeleton_npy/train')
os.mkdir('human_act_skeleton_npy/valid')

In [5]:
# for train npy
max_frames = 100
SAVE_DIR = 'human_act_skeleton_npy/'

with mp_pose.Pose( min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    all_video_train_pose = []
    for i, elem in tqdm(train_df.iterrows() , total = len(train_df)):
        cap = cv2.VideoCapture(
        elem['file_path'] 
        )

        # all skeleton landmark가 다 담긴다 
        video_skeleton=[]
        while True:
            ret, frame  = cap.read()
            if not ret:
                break
            frame, results =mediapipe_detection(frame,pose)

            # results가 존재한다면 -> landmark를 넣는다. (NoneType error를 막기 위한 코드!)
            if results.pose_landmarks is not None:
                video_skeleton.append(extract_keypoints(results))
        
        label = elem['label']

        video_skeleton = np.array(video_skeleton[-max_frames:])
        file_name = SAVE_DIR+f'/train/{label}_{i}.npy'
        np.save(file_name, video_skeleton)
        


100%|██████████| 861/861 [2:21:40<00:00,  9.87s/it]  


In [6]:
from glob import glob
print(len(glob(SAVE_DIR+'/train/*'))) 

861


In [7]:
# for valid npy
max_frames = 100
SAVE_DIR = 'human_act_skeleton_npy/'

with mp_pose.Pose( min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    for i, elem in tqdm(valid_df.iterrows() , total = len(valid_df)):
        cap = cv2.VideoCapture(
        elem['file_path'] 
        )

        # all skeleton landmark가 다 담긴다 
        video_skeleton=[]
        while True:
            ret, frame  = cap.read()
            if not ret:
                break
            frame, results =mediapipe_detection(frame,pose)

            # results가 존재한다면 -> landmark를 넣는다. (NoneType error를 막기 위한 코드!)
            if results.pose_landmarks is not None:
                video_skeleton.append(extract_keypoints(results))
        
        label = elem['label']
        
        video_skeleton = np.array(video_skeleton[-max_frames:])
        file_name = SAVE_DIR+f'/valid/{label}_{i}.npy'
        np.save(file_name, video_skeleton)
        

100%|██████████| 306/306 [49:08<00:00,  9.64s/it] 


In [8]:
print(len(glob(SAVE_DIR+'/valid/*'))) 

306


In [14]:

import numpy as np
from glob import glob
mode = 'valid'
data= glob(
            f'human_act_skeleton_npy/{mode}/*.npy'
        )

cnt=[]
for i in data:
    tmp = np.load(i)
    if(tmp.shape[0] !=100) or len(tmp.shape)!=2:
        cnt.append(i)

print(cnt)

['human_act_skeleton_npy/valid\\fall_10.npy', 'human_act_skeleton_npy/valid\\fall_32.npy', 'human_act_skeleton_npy/valid\\fall_34.npy', 'human_act_skeleton_npy/valid\\fall_38.npy']


In [15]:
len(data)

306

In [16]:
import os

for i in cnt:
    dir_path = f'human_act_skeleton_npy/{mode}'
    tmp =dir_path+"/" +i.split('\\')[1]
    # os.remove()
    print(tmp)
    if os.path.isfile(tmp):
        os.remove(tmp)
        print("ok")
        
data= glob(
    
            f'human_act_skeleton_npy/{mode}/*.npy'

    )
len(data)

human_act_skeleton_npy/valid/fall_10.npy
ok
human_act_skeleton_npy/valid/fall_32.npy
ok
human_act_skeleton_npy/valid/fall_34.npy
ok
human_act_skeleton_npy/valid/fall_38.npy
ok


302

215